In [1025]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [1026]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"SSRM"}, 
             {'symbol':"LRN"}, 
             {'symbol':"UNFI", 'basis':22.360}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':100.482}, 
             {'symbol':"EZPW", 'basis':15.460}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':62.588}, 
             {'symbol':"ITRN", 'basis':35.710}, 
             {'symbol':"CRDO", 'basis':0.010}, 
             {'symbol':"PYPL", 'basis':69.250}, 
             {'symbol':"ALL", 'basis':195.919}, 
             {'symbol':"LC", 'basis':9.703}, 
             {'symbol':"QTWO"}, 
             {'symbol':"CLS", 'basis':21.230}, 
             {'symbol':"CCL", 'basis':16.147}, 
             {'symbol':"AGX", 'basis':7.920}, 
             {'symbol':"POWL", 'basis':186.813}, 
             {'symbol':"PPC", 'basis':45.089}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE"}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':76.148}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA", 'basis':101.620}, 
             {'symbol':"TWLO", 'basis':91.565}, 
             {'symbol':"APP", 'basis':221.323}, 
             {'symbol':"TMUS", 'basis':227.463}, 
             {'symbol':"STRL"}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':45.855}, 
             {'symbol':"CAAP", 'basis':21.570}
             ]

# Brokerage
brokerage_funds = [{'symbol':"SSRM"}, 
                   {'symbol':'BCS'}, # UK - Financials
                   {'symbol':'PUK'}, # Hong Kong - Financials
                   {'symbol':'FINV'}, # China - Financials
                   {'symbol':'PSIX'}, # U.S. - Industrials
                   {'symbol':'NGD'}, # Canada - Materials (Gold)
                   {'symbol':'GFI'}, # South Africa - Materials (Gold)
                   {'symbol':"CRDO", 'basis':38.629}, 
                   {'symbol':"COMM"}, 
                   {'symbol':'BKTI'}, # United States - Communication Equipment
                   {'symbol':"EAT", 'basis':135.74}, 
                   {'symbol':"UNFI", 'basis':23.000}, 
                   {'symbol':"EZPW", 'basis':17.715}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"WFC", 'basis':68.586}, 
                   {'symbol':"PYPL", 'basis':69.600}, 
                   {'symbol':"CLS", 'basis':74.924}, 
                   {'symbol':"CCL", 'basis':20.112}, 
                   {'symbol':"ATGE", 'basis':86.385}, 
                   {'symbol':"SKYW", 'basis':84.188}, 
                   {'symbol':"OKTA", 'basis':104.950}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   {'symbol':"UBER"},
                   {'symbol':"FBTC", 'basis':75.244, 'is_etf': True}, 
                   {'symbol':"NVDA", 'basis':95.19}
                   ]

In [1027]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [1028]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=1, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [1029]:
brokerage_df[brokerage_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
7,CRDO,38.629,91.92,55.93,6.87,60.36,291.62,21.070,0.0000,Sell
10,EAT,135.740,180.04,148.02,38.17,156.25,24.88,30.660,0.0000,Sell
11,UNFI,23.000,21.69,24.76,24.76,26.17,inf,0.797,0.0000,Hold
12,EZPW,17.715,13.32,14.73,23.99,15.29,11.13,0.950,0.0000,Hold
13,ARQT,15.996,14.33,13.65,13.65,14.69,inf,11.550,0.0000,Hold
14,WFC,68.586,78.12,69.27,101.10,71.57,13.56,1.510,0.0212,Sell
15,PYPL,69.600,73.58,65.94,57.61,68.30,16.18,3.458,0.0000,Sell
16,CLS,74.924,143.78,93.91,43.17,100.41,38.08,9.990,0.0000,Sell
17,CCL,20.112,25.69,20.31,20.40,21.28,15.51,3.507,0.0000,Sell
18,ATGE,86.385,126.30,113.83,88.69,119.24,21.66,3.211,0.0000,Sell


In [1030]:
brokerage_df[brokerage_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,SSRM,0.0,12.36,11.03,14.06,11.60,31.78,0.834,0.0000,Sell
1,BCS,0.0,18.25,15.97,45.99,16.34,8.71,0.620,0.0240,Sell
2,PUK,0.0,24.83,21.58,28.75,22.04,14.27,1.792,0.0193,Sell
3,FINV,0.0,9.82,8.45,21.10,8.90,6.70,1.108,0.0299,Sell
4,PSIX,0.0,64.41,33.53,22.53,37.05,17.24,16.260,0.0000,Sell
5,NGD,0.0,4.84,3.93,2.82,4.12,27.39,3.699,0.0000,Sell
6,GFI,0.0,23.84,22.20,16.21,23.04,17.67,4.197,0.0223,Sell
8,COMM,0.0,7.33,4.81,4.81,5.22,inf,30.360,0.0000,Sell
9,BKTI,0.0,54.50,41.64,31.00,46.14,19.52,5.430,0.0000,Sell
22,UBER,0.0,91.62,79.67,48.76,82.91,14.93,8.110,0.0000,Sell


In [1031]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=.95, vwap_days=63)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [1032]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
2,UNFI,22.360,21.69,23.52,23.52,26.17,inf,0.797,0.0000,Hold
4,EAT,100.482,178.64,140.29,37.88,155.92,24.88,30.660,0.0000,Sell
5,EZPW,15.460,13.32,13.99,23.99,15.29,11.13,0.950,0.0000,Hold
6,ARQT,15.932,14.33,12.97,12.97,14.69,inf,11.550,0.0000,Hold
7,WFC,62.588,78.12,65.81,101.10,71.57,13.56,1.510,0.0212,Sell
8,ITRN,35.710,36.05,33.32,30.56,36.38,12.85,3.713,0.0562,Hold
9,CRDO,0.010,91.92,53.13,6.87,60.36,291.62,21.070,0.0000,Sell
10,PYPL,69.250,73.58,62.64,57.61,68.30,16.18,3.458,0.0000,Sell
11,ALL,195.919,196.06,186.23,194.11,201.56,13.54,2.584,0.0205,Hold
12,LC,9.703,11.56,9.48,13.88,10.50,25.41,0.936,0.0000,Sell


In [1033]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,SSRM,0.0,12.36,10.48,14.06,11.60,31.78,0.834,0.0000,Sell
1,LRN,0.0,144.46,133.27,82.91,146.82,23.05,4.517,0.0000,Hold
3,MFC,0.0,31.31,28.54,35.99,30.92,16.31,1.591,0.0410,Sell
13,QTWO,0.0,95.31,77.09,77.09,84.50,inf,10.110,0.0000,Sell
19,SYF,0.0,64.76,49.79,103.68,54.83,8.66,1.545,0.0190,Sell
20,ATGE,0.0,126.30,108.14,88.69,119.24,21.66,3.211,0.0000,Sell
21,BRK-B,0.0,493.66,478.58,632.30,516.26,13.00,1.608,0.0000,Hold
22,SFM,0.0,166.47,150.78,44.96,165.42,37.29,12.610,0.0000,Sell
24,BLBD,0.0,43.19,34.21,26.27,37.55,13.32,6.960,0.0000,Sell
25,GM,0.0,48.99,43.91,129.85,47.85,6.78,0.720,0.0124,Sell


In [1034]:
# Research new investments
watch_list = [{'symbol':'BSAC'}, # Chile - Latin America - Financials
              {'symbol':'KARO'}, # Singapore - Technology - SaaS
              {'symbol':'QFIN'}, # China - FinTech
              {'symbol':'TSM'}, # Taiwan - Semiconductors
              {'symbol':'CCU'}, # Chile - Consumer Staples - beverage distributor
              {'symbol':'ABEV'}, # Brazil - Financials - but distributes beers?
              {'symbol':'TIMB'}, # Brazil - Communications - Telecom (mobile networking)
              {'symbol':'ASR'}, # Mexico - Industrials - Airport operations in South America
              {'symbol':'RYAAY'}, # Ireland - Industrials - budget airline
              {'symbol':'AMX'}, # Mexico - Communications - mobile network
              {'symbol':'ERIC'}, # Sweden - Communications
              {'symbol':'STX'} # Ireland - Technology
             ] 
watch_table = build_analysis_table(watch_list, api_key, margin_of_safety=1, vwap_days=63)
internationals = build_analysis_table(watch_list, api_key, margin_of_safety=0.95, vwap_days=126)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

internationals_df = pd.DataFrame(internationals, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [1035]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,24.45,23.56,28.60,24.18,10.42,2.405,0.0570,Sell
1,KARO,0,47.77,47.10,17.87,50.27,28.69,8.540,0.0229,Hold
2,QFIN,0,45.80,40.54,77.25,42.77,6.66,1.810,0.0301,Sell
3,TSM,0,220.10,171.11,96.68,177.63,25.00,7.110,0.0154,Sell
4,CCU,0,12.71,14.29,17.32,14.75,13.02,1.419,0.0365,Hold
5,ABEV,0,2.40,2.39,2.51,2.45,14.24,2.206,0.0631,Hold
6,TIMB,0,19.19,16.58,20.09,16.96,14.91,2.098,0.0803,Sell
7,ASR,0,306.30,304.29,279.92,313.13,12.82,3.203,0.0815,Hold
8,RYAAY,0,57.12,49.65,43.42,50.96,16.41,3.616,0.0170,Sell
9,AMX,0,17.00,16.06,10.39,16.48,35.48,2.588,0.0321,Sell


In [1036]:
internationals_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,24.45,21.56,28.60,23.24,10.42,2.405,0.0570,Sell
1,KARO,0,47.77,43.62,17.87,48.71,28.69,8.540,0.0229,Hold
2,QFIN,0,45.80,38.45,77.25,42.62,6.66,1.810,0.0301,Sell
3,TSM,0,220.10,173.26,96.68,189.31,25.00,7.110,0.0154,Sell
4,CCU,0,12.71,13.02,17.32,14.12,13.02,1.419,0.0365,Hold
5,ABEV,0,2.40,2.03,2.51,2.19,14.24,2.206,0.0631,Sell
6,TIMB,0,19.19,14.07,20.09,15.13,14.91,2.098,0.0803,Sell
7,ASR,0,306.30,275.09,279.92,298.42,12.82,3.203,0.0815,Sell
8,RYAAY,0,57.12,45.07,43.42,48.67,16.41,3.616,0.0170,Sell
9,AMX,0,17.25,14.63,10.54,15.79,35.48,2.588,0.0321,Sell


In [1037]:
# Research new investments
mag7_test = [{'symbol':'MSFT'}, 
             {'symbol':'AAPL'}, 
             {'symbol':'NVDA'}, 
             {'symbol':'AMZN'}, 
             {'symbol':'GOOG'}, 
             {'symbol':'GOOGL'}, 
             {'symbol':'META'}, 
             {'symbol':'BRK-B'}, 
             {'symbol':'TSLA'}
             ] 
mag7_test_table = build_analysis_table(mag7_test, api_key, vwap_days=63)

# Create watch list data frame
mag7_df = pd.DataFrame(mag7_test_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [1038]:
mag7_test_table

[['MSFT', 0, 490.11, 373.16, 141.02, 424.94, 37.59, 11.02, 0.0068, 'Sell'],
 ['AAPL', 0, 200.3, 177.25, 31.25, 204.75, 31.36, 44.94, 0.0052, 'Hold'],
 ['NVDA', 0, 147.9, 104.7, 19.63, 121.86, 46.51, 41.84, 0.0003, 'Sell'],
 ['AMZN', 0, 212.77, 170.77, 79.32, 197.04, 33.9, 7.28, 0, 'Sell'],
 ['GOOG', 0, 167.74, 145.07, 94.51, 166.95, 18.53, 5.83, 0.0051, 'Sell'],
 ['GOOGL', 0, 166.77, 143.41, 92.11, 165.1, 18.46, 6.09, 0.0051, 'Sell'],
 ['META', 0, 712.2, 522.98, 265.31, 604.22, 26.66, 9.27, 0.0031, 'Sell'],
 ['BRK-B', 0, 493.66, 453.39, 632.3, 516.26, 13.0, 1.608, 0, 'Hold'],
 ['TSLA', 0, 340.47, 255.27, 37.78, 303.24, 200.39, 13.9, 0, 'Sell']]

In [1039]:
mag7_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,MSFT,0,490.11,373.16,141.02,424.94,37.59,11.020,0.0068,Sell
1,AAPL,0,200.30,177.25,31.25,204.75,31.36,44.940,0.0052,Hold
2,NVDA,0,147.90,104.70,19.63,121.86,46.51,41.840,0.0003,Sell
3,AMZN,0,212.77,170.77,79.32,197.04,33.90,7.280,0.0000,Sell
4,GOOG,0,167.74,145.07,94.51,166.95,18.53,5.830,0.0051,Sell
5,GOOGL,0,166.77,143.41,92.11,165.10,18.46,6.090,0.0051,Sell
6,META,0,712.20,522.98,265.31,604.22,26.66,9.270,0.0031,Sell
7,BRK-B,0,493.66,453.39,632.30,516.26,13.00,1.608,0.0000,Hold
8,TSLA,0,340.47,255.27,37.78,303.24,200.39,13.900,0.0000,Sell


In [1040]:
# Machine Learning

In [1041]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [1042]:
# Feature selection and normalization

In [1043]:
# Build model

In [1044]:
# Train model

In [1045]:
# Evaluate model

In [1046]:
# Refine model through hyperparameter tuning